# APIM ❤️ OpenAI

## Built-in logging lab
![flow](../../images/built-in-logging.gif)

Playground to try routing to a backend based on Azure OpenAI model and version.

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Create the Azure Resource Group](#1)
- [2️⃣ Create deployment using 🦾 Bicep](#2)
- [3️⃣ Get the deployment outputs](#3)
- [🧪 Test the API using a direct HTTP call](#requests)
- [🧪 Test the API using the Azure OpenAI Python SDK](#sdk)
- [🗑️ Clean up resources](#clean)

### Backlog
- Improve the notebook

### Prerequisites
- [Python 3.8 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- The ```mock_webapps``` variable sets the list of deployed Web Apps for the mocking functionality. Clean the ```openai_resources``` list to simulate the OpenAI behaviour with the mocking service.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models) 

In [13]:
import os
import json
import datetime
import requests

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
index = '2'
resource_group_name = f"lab-{deployment_name}{index}" # change the name to match your naming style
resource_group_location = "westeurope"

# Define two OpenAI model and version combinations
# https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#gpt-35
# Please note that availability of models and versions is variable and that you may need to adjust the model and version names to match the available models and versions in your Azure subscription.
# For this lab, we are using the following combinations based on PayGo availability on June 20, 2024:
#
#   1) GPT-3.5 Turbo 1106: France Central, Sweden Central
#   2) GPT-3.5 Turbo 0125: North Central US, South Central US

openai_model_1_name = "gpt-35-turbo"
openai_model_1_version = "1106"
openai_deployment_1_name = f"{openai_model_1_name}-{openai_model_1_version}"
openai_resources_1 = [ {"name": "oai-francecentral", "location": "francecentral"}, {"name": "oai-swedencentral", "location": "swedencentral"} ] # list of OpenAI resources to deploy. Clear this list to use only the mock resources

openai_model_2_name = "gpt-35-turbo"
openai_model_2_version = "0125"
openai_deployment_2_name = f"{openai_model_2_name}-{openai_model_2_version}"
openai_resources_2 = [ {"name": "oai-northcentralus", "location": "northcentralus"}, {"name": "oai-southcentralus", "location": "southcentralus"} ] # list of OpenAI resources to deploy. Clear this list to use only the mock resources

# Define Azure OpenAI resources
openai_resources_sku = "S0"
openai_api_version = "2024-02-01"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/' + openai_api_version + '/inference.json'

# Define Azure API Management
apim_resource_name = "apim"
apim_resource_location = "westeurope"
apim_resource_sku = "Basicv2"

# Define the Azure OpenAI backends and backend pools per Azure OpenAI model and version
openai_backend_pool_1 = f"oai-backend-pool-{openai_deployment_1_name}"
openai_backend_pool_2 = f"oai-backend-pool-{openai_deployment_2_name}"

log_analytics_name = "workspace"
app_insights_name = 'insights'


<a id='1'></a>
### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [14]:
# type: ignore
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location} 

if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())

✅ Azure Resource Group  lab-model-routing1  created ⌚  16:15:51.170076


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [11]:
if len(openai_resources_1) > 0:
    backend_id_1 = openai_backend_pool_1 if len(openai_resources_1) > 1 else openai_resources_1[0].get("name")
if len(openai_resources_2) > 0:
    backend_id_2 = openai_backend_pool_2 if len(openai_resources_2) > 1 else openai_resources_2[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_template_xml = policy_xml_file.read()
    policy_xml = policy_template_xml.replace("{backend-id-1}", backend_id_1).replace("{backend-id-2}", backend_id_2)
    policy_xml_file.close()
open("policy.xml", 'w').write(policy_xml)

bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "openAIBackendPoolName_1": { "value": openai_backend_pool_1 },
    "openAIBackendPoolName_2": { "value": openai_backend_pool_2 },
    "openAIConfig_1": { "value": openai_resources_1 },
    "openAIConfig_2": { "value": openai_resources_2 },
    "openAIDeploymentName_1": { "value": openai_deployment_1_name },
    "openAIDeploymentName_2": { "value": openai_deployment_2_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName_1": { "value": openai_model_1_name },
    "openAIModelName_2": { "value": openai_model_2_name },
    "openAIModelVersion_1": { "value": openai_model_1_version },
    "openAIModelVersion_2": { "value": openai_model_2_version },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name},
    "apimResourceLocation": { "value": apim_resource_location},
    "apimSku": { "value": apim_resource_sku},
    "logAnalyticsName": { "value": log_analytics_name },
    "applicationInsightsName": { "value": app_insights_name },
    "index": { "value": index}
  }
}
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"

open("policy.xml", 'w').write(policy_template_xml)


ERROR: {"code": "ResourceGroupBeingDeleted", "message": "The resource group 'lab-model-routing' is in deprovisioning state and cannot perform this operation."}


1024

<a id='3'></a>
### 3️⃣ Get the deployment outputs

We are now at the stage where we only need to retrieve the gateway URL and the subscription before we are ready for testing.

In [15]:
# type: ignore
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = deployment_stdout.n

# type: ignore
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = deployment_stdout.n
print("👉🏻 API Gateway URL: ", apim_resource_gateway_url)

# type: ignore
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.logAnalyticsWorkspaceId.value -o tsv
workspace_id = deployment_stdout.n
print("👉🏻 Workspace ID: ", workspace_id)

# type: ignore
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.applicationInsightsAppId.value -o tsv
app_id = deployment_stdout.n
print("👉🏻 App ID: ", app_id)



^C
👉🏻 API Gateway URL:  
👉🏻 Workspace ID:  ERROR: (DeploymentNotFound) Deployment 'model-routing' could not be found.
Code: DeploymentNotFound
Message: Deployment 'model-routing' could not be found.
👉🏻 App ID:  ERROR: (DeploymentNotFound) Deployment 'model-routing' could not be found.
Code: DeploymentNotFound
Message: Deployment 'model-routing' could not be found.


<a id='requests'></a>
### 🧪 Test the API using a direct HTTP call
Requests is an elegant and simple HTTP library for Python that will be used here to make raw API requests and inspect the responses.

In [32]:
import time
runs = 2
sleep_time_ms = 1000

# GPT-3.5 Turbo 1106
for i in range(runs):
    print("▶️ GPT-3.5 Turbo 1106 Run: ", i+1)
    if len(openai_resources_1) > 0:
        messages={"messages":[
            {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
            {"role": "user", "content": "Can you tell me the time, please?"}
        ]}

    url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_1_name + "/chat/completions?api-version=" + openai_api_version        
    response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = messages)
    print("status code: ", response.status_code)
    print("headers ", response.headers)
    print("x-ms-region: ", response.headers.get("x-ms-region")) # this header is useful to determine the region of the backend that served the request
    if (response.status_code == 200):
        data = json.loads(response.text)
        print("response: ", data.get("choices")[0].get("message").get("content"))
    else:
        print(response.text)
    time.sleep(sleep_time_ms/1000)

# GPT-3.5 Turbo 0125
for i in range(runs):
    print("▶️ GPT-3.5 Turbo 0125 Run: ", i+1)
    if len(openai_resources_1) > 0:
        messages={"messages":[
            {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
            {"role": "user", "content": "Can you tell me the time, please?"}
        ]}

    url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_2_name + "/chat/completions?api-version=" + openai_api_version        
    response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = messages)
    print("status code: ", response.status_code)
    print("headers ", response.headers)
    print("x-ms-region: ", response.headers.get("x-ms-region")) # this header is useful to determine the region of the backend that served the request
    if (response.status_code == 200):
        data = json.loads(response.text)
        print("response: ", data.get("choices")[0].get("message").get("content"))
    else:
        print(response.text)
    time.sleep(sleep_time_ms/1000)

▶️ Run:  1
status code:  200
headers  {'Content-Length': '930', 'Content-Type': 'application/json', 'Date': 'Thu, 20 Jun 2024 19:10:24 GMT', 'Access-Control-Allow-Origin': '*', 'Cache-Control': 'no-cache, must-revalidate', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains; preload', 'apim-request-id': '266cad86-b42f-44d8-99c5-04940e89eee0', 'X-Content-Type-Options': 'nosniff', 'x-ms-region': 'France Central', 'x-ratelimit-remaining-requests': '19', 'x-ratelimit-remaining-tokens': '18680', 'x-accel-buffering': 'no', 'x-ms-rai-invoked': 'true', 'X-Request-ID': '141a9030-dfa7-404d-845e-fcc3c45ba082', 'x-ms-client-request-id': 'Not-Set', 'azureml-model-session': 'd027-20240603054521', 'Request-Context': 'appId=cid-v1:c844dffc-e3d9-471d-828a-78852087a74d'}
x-ms-region:  France Central
response:  Oh, I'm so sorry, I don't have the capability to tell time. You'll have to consult that fancy, outdated device on your wrist called a watch.
▶️ Run:  2
status code:  200
headers  {'C

<a id='sdk'></a>
### 🧪 Test the API using the Azure OpenAI Python SDK
OpenAPI provides a widely used [Python library](https://github.com/openai/openai-python). The library includes type definitions for all request params and response fields. The goal of this test is to assert that APIM can seamlessly proxy requests to OpenAI without disrupting its functionality.
- Note: run ```pip install openai``` in a terminal before executing this step.

In [34]:
import time
runs = 2
sleep_time_ms = 1000

# GPT-3.5 Turbo 1106
for i in range(runs):
    print("▶️ GPT-3.5 Turbo 1106 Run: ", i+1)
    from openai import AzureOpenAI
    if len(openai_resources_1) > 0:
        messages=[
            {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
            {"role": "user", "content": "Can you tell me the time, please?"}
        ]
        
    client = AzureOpenAI(
        azure_endpoint=apim_resource_gateway_url,
        api_key=apim_subscription_key,
        api_version=openai_api_version
    )
    response = client.chat.completions.create(model=openai_deployment_1_name, messages=messages)
    print(response.choices[0].message.content)
    time.sleep(sleep_time_ms/1000)

# GPT-3.5 Turbo 0125
for i in range(runs):
    print("▶️ GPT-3.5 Turbo 0125 Run: ", i+1)
    from openai import AzureOpenAI
    if len(openai_resources_2) > 0:
        messages=[
            {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
            {"role": "user", "content": "Can you tell me the time, please?"}
        ]
        
    client = AzureOpenAI(
        azure_endpoint=apim_resource_gateway_url,
        api_key=apim_subscription_key,
        api_version=openai_api_version
    )
    response = client.chat.completions.create(model=openai_deployment_2_name, messages=messages)
    print(response.choices[0].message.content)
    time.sleep(sleep_time_ms/1000)


▶️ GPT-3.5 Turbo 1106 Run:  1
Sure, let me just consult my watch that doesn't exist.
▶️ GPT-3.5 Turbo 1106 Run:  2
Sure, let me just conjure a clock out of thin air and tell you the time. Oh wait, I can't do that. Why don't you look at a clock or your phone like a normal person?
▶️ GPT-3.5 Turbo 0125 Run:  1
Sure, have you considered looking at a clock or your phone? It tends to have that information readily available.
▶️ GPT-3.5 Turbo 0125 Run:  2
Oh, I’m so glad you asked me that! *eye roll* Did you know that there are devices specifically designed for determining the time? It’s a whole new world out there!


<a id='kql'></a>
### 🔍 Analyze Application Insights requests

With this query you can get the request and response details including the prompt and the OpenAI completion. It also returns token counters.

In [36]:
import pandas as pd

query = "\"" + "requests  \
| project timestamp, duration, customDimensions \
| extend duration = round(duration, 2) \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend apiName = tostring(parsedCustomDimensions.['API Name']) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription Name']) \
| extend userAgent = tostring(parsedCustomDimensions.['Request-User-agent']) \
| extend request_json = tostring(parsedCustomDimensions.['Request-Body']) \
| extend request = parse_json(request_json) \
| extend model = tostring(request.['model']) \
| extend messages = tostring(request.['messages']) \
| extend region = tostring(parsedCustomDimensions.['Response-x-ms-region']) \
| extend remainingTokens = tostring(parsedCustomDimensions.['Response-x-ratelimit-remaining-tokens']) \
| extend remainingRequests = tostring(parsedCustomDimensions.['Response-x-ratelimit-remaining-requests']) \
| extend response_json = tostring(parsedCustomDimensions.['Response-Body']) \
| extend response = parse_json(response_json) \
| extend promptTokens = tostring(response.['usage'].['prompt_tokens']) \
| extend completionTokens = tostring(response.['usage'].['completion_tokens']) \
| extend totalTokens = tostring(response.['usage'].['total_tokens']) \
| extend completion = tostring(response.['choices'][0].['message'].['content']) \
| project timestamp, apiName, apimSubscription, duration, userAgent, model, messages, completion, region, promptTokens, completionTokens, totalTokens, remainingTokens, remainingRequests \
| order by timestamp desc" + "\""

result_stdout = ! az monitor app-insights query --app {app_id} --analytics-query {query} # type: ignore
result = json.loads(result_stdout.n)

table = result.get('tables')[0]
pd.DataFrame(table.get("rows"), columns=[col.get("name") for col in table.get('columns')])


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

<a id='portal'></a>
### 🔍 Open the workbook in the Azure Portal

Go to the application insights resource and under the Monitoring section select the Workbooks blade. You should see the OpenAI Usage Analysis workbook with the above query and some others to check token counts, performance, failures, etc.

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.